<a href="https://colab.research.google.com/github/DineshKesawamoorthy/nlp/blob/main/sentence_similarity/similarity_using_sentense_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

In [5]:
model_name="sentence-transformers/bert-base-nli-cls-token"

In [6]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/395 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [8]:
text="Please follow our blog to see more information about new features, tips and tricks, and featured notebooks such as Analyzing a Bank Failure with Colab."

In [11]:
tokens=tokenizer.encode_plus(text,max_length=128,truncation=True,padding='max_length',return_tensors='pt')

In [12]:
output=model(**tokens)

In [13]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3122,  0.5022,  0.1566,  ..., -0.4626, -0.7626, -0.4060],
         [-0.0714,  0.3613,  0.5192,  ..., -0.0933, -0.9233, -0.3021],
         [ 0.3376,  0.2263,  0.5899,  ..., -0.0443, -0.4724, -0.4827],
         ...,
         [-0.1530,  0.1472,  0.1972,  ..., -0.0121, -0.6203, -0.4213],
         [-0.1756,  0.1143,  0.2650,  ...,  0.0160, -0.6303, -0.3837],
         [-0.1132,  0.1133,  0.3309,  ...,  0.0590, -0.5916, -0.3910]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-5.4622e-01, -3.7354e-01,  8.9649e-01, -1.6514e-01, -1.1777e-01,
         -3.0486e-01, -2.1445e-02,  2.7738e-01,  9.4004e-01, -9.8190e-01,
          2.0353e-01, -7.8980e-01,  9.8908e-01, -9.0278e-01,  9.3163e-01,
         -1.5600e-01,  5.5747e-01, -2.0699e-01,  1.1115e-01,  8.1613e-01,
          7.8582e-01,  8.9778e-01,  6.8009e-01,  1.6048e-01,  7.2027e-02,
         -8.8313e-01, -2.1479e-01,  9.1384e-01,  8.8966e-01,  7.804

In [16]:
embeddings=output.last_hidden_state
embeddings

tensor([[[-0.3122,  0.5022,  0.1566,  ..., -0.4626, -0.7626, -0.4060],
         [-0.0714,  0.3613,  0.5192,  ..., -0.0933, -0.9233, -0.3021],
         [ 0.3376,  0.2263,  0.5899,  ..., -0.0443, -0.4724, -0.4827],
         ...,
         [-0.1530,  0.1472,  0.1972,  ..., -0.0121, -0.6203, -0.4213],
         [-0.1756,  0.1143,  0.2650,  ...,  0.0160, -0.6303, -0.3837],
         [-0.1132,  0.1133,  0.3309,  ...,  0.0590, -0.5916, -0.3910]]],
       grad_fn=<NativeLayerNormBackward0>)

In [18]:
embeddings.shape

torch.Size([1, 128, 768])

In [21]:
attention_mask=tokens['attention_mask']
attention_mask.shape

torch.Size([1, 128])

In [25]:
attention_mask.unsqueeze(-1).shape

torch.Size([1, 128, 1])

In [27]:
attention_mask.unsqueeze(-1).expand(embeddings.shape).shape

torch.Size([1, 128, 768])

In [28]:
mask=attention_mask.unsqueeze(-1).expand(embeddings.shape).float()

# Masked Embeddings

In [30]:
masked_embeddings=embeddings* mask
masked_embeddings.shape

torch.Size([1, 128, 768])

In [31]:
masked_embeddings

tensor([[[-0.3122,  0.5022,  0.1566,  ..., -0.4626, -0.7626, -0.4060],
         [-0.0714,  0.3613,  0.5192,  ..., -0.0933, -0.9233, -0.3021],
         [ 0.3376,  0.2263,  0.5899,  ..., -0.0443, -0.4724, -0.4827],
         ...,
         [-0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000]]],
       grad_fn=<MulBackward0>)

In [34]:
# finding average of last hidden state
summed= torch.sum(masked_embeddings,1)
summed.shape

torch.Size([1, 768])

In [36]:
counts=torch.clamp(mask.sum(1),min=1e-9)
counts.shape

torch.Size([1, 768])

In [38]:
mean_of_last_hidden_state=summed/counts
mean_of_last_hidden_state.shape

torch.Size([1, 768])

# Preparing Embeddings for multiple Sentences

In [67]:
sentences=["TPU v2 runtime is now available",
"L4 runtime is now available for paid users",
"New distributed fine-tuning Gemma tutorial on TPUs (GitHub)",
"Symbol rename is now supported with keyboard shortcut F2",
"Fixed bug causing inability to re-upload deleted files",
 "TPU v2 runtime is now active"]
sentences

['TPU v2 runtime is now available',
 'L4 runtime is now available for paid users',
 'New distributed fine-tuning Gemma tutorial on TPUs (GitHub)',
 'Symbol rename is now supported with keyboard shortcut F2',
 'Fixed bug causing inability to re-upload deleted files',
 'TPU v2 runtime is now active']

In [68]:
tokens={
    "input_ids":[],
    "attention_mask":[]
}

In [69]:
for sentence in sentences:
  new_tokens=tokenizer.encode_plus(text,max_length=128,truncation=True,padding='max_length',return_tensors='pt')
  tokens['input_ids'].append(new_tokens['input_ids'][0])
  tokens['attention_mask'].append(new_tokens['attention_mask'][0])

In [70]:
# reformat list of tensors into single tensor
tokens['input_ids']=torch.stack(tokens['input_ids'])
tokens['attention_mask']=torch.stack(tokens['attention_mask'])

In [71]:
tokens['input_ids'].shape

torch.Size([6, 128])

In [72]:
outputs=model(**tokens)

In [73]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [74]:
embeddings=outputs.last_hidden_state
embeddings.shape

torch.Size([6, 128, 768])

In [75]:
attention_mask=tokens['attention_mask']
attention_mask.shape

torch.Size([6, 128])

In [76]:
mask=attention_mask.unsqueeze(-1).expand(embeddings.shape).float()
mask.shape

torch.Size([6, 128, 768])

In [77]:
masked_embeddings=embeddings* mask
masked_embeddings.shape

torch.Size([6, 128, 768])

In [78]:
counts=torch.clamp(mask.sum(1),min=1e-9)
counts.shape

torch.Size([6, 768])

In [79]:
mean_of_last_hidden_state=summed/counts
mean_of_last_hidden_state.shape

torch.Size([6, 768])

# Calculating Similarity

In [80]:
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
mean_of_last_hidden_state=mean_of_last_hidden_state.detach().numpy()

cosine_similarity([mean_of_last_hidden_state[0]],mean_of_last_hidden_state[1:])

array([[0.9999999, 0.9999999, 0.9999999, 0.9999999, 0.9999999]],
      dtype=float32)

# Same Task using Sentence-Transformers Library

In [84]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [82]:
sentences=["TPU v2 runtime is now available",
"L4 runtime is now available for paid users",
"New distributed fine-tuning Gemma tutorial on TPUs (GitHub)",
"Symbol rename is now supported with keyboard shortcut F2",
"Fixed bug causing inability to re-upload deleted files",
 "TPU v2 runtime is now active"]
sentences

['TPU v2 runtime is now available',
 'L4 runtime is now available for paid users',
 'New distributed fine-tuning Gemma tutorial on TPUs (GitHub)',
 'Symbol rename is now supported with keyboard shortcut F2',
 'Fixed bug causing inability to re-upload deleted files',
 'TPU v2 runtime is now active']

In [86]:
model_name="bert-base-nli-cls-token"

In [87]:
from sentence_transformers import SentenceTransformer

In [88]:
model= SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.60k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [90]:
embeddings=model.encode(sentences)
embeddings

array([[ 0.06882696, -0.56251556,  0.54714644, ..., -0.08211193,
        -0.1139864 , -0.23310672],
       [-0.27272668, -0.7018617 ,  0.74937975, ..., -0.1718269 ,
        -0.7057901 , -0.15645607],
       [-0.40502045, -0.44330147,  0.07816456, ..., -0.5007229 ,
        -0.48896366,  0.18308051],
       [-1.160834  , -0.11962634,  0.10682586, ..., -0.14017826,
        -0.43472454, -0.00406605],
       [-0.02385116,  0.28638878,  1.3774637 , ...,  0.46538323,
        -1.2008551 , -0.3983584 ],
       [ 0.14035167, -0.449393  ,  0.88668203, ..., -0.13413827,
        -0.2817056 , -0.29315606]], dtype=float32)

In [91]:
embeddings.shape

(6, 768)

In [92]:
from sklearn.metrics.pairwise import cosine_similarity

In [100]:
similarity_scores=cosine_similarity([embeddings[0]],embeddings[1:])
similarity_scores # last sentence will have the highest score

array([[0.62496376, 0.5803967 , 0.70578086, 0.34281382, 0.9424119 ]],
      dtype=float32)